# 상장기업정보 수집 하기

In [1]:
import time
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
def info_extraction(stock_list):
    page_result = []
    for stock in stock_list:
        stock_name = stock.select_one(".vmiddle.legend")['alt']
        company_name = stock.select_one("#companysum").text
        code = stock.select_one("#companysum")['onclick'].split("'")[1]
        code = f"{code:0<6}"
        industry = stock.select_one("td:nth-child(2)").text 
        main_products = stock.select_one("td:nth-child(3)").text
        listing_date = stock.select_one("td:nth-child(4)").text
        fiscal_month = stock.select_one("td:nth-child(5)").text
        ceo_name = stock.select_one("td:nth-child(6)").text
        website = stock.select_one(".btn.ico.homepage")['href'] if stock.select_one(".btn.ico.homepage") != None else "없음"
        region = stock.select_one("td:nth-child(8)").text
        values = (stock_name, company_name, code, industry, main_products, listing_date, fiscal_month, ceo_name, website, region)
        page_result.append(values)
    return page_result

In [3]:
url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
payload = dict(method="searchCorpList",pageIndex=1,currentPageSize=100,orderMode=3,orderStat="D",searchType=13, fiscalYearEnd="all", location="all")
r = requests.post(url, data = payload)
print(r.url)
print(r.status_code)
soup = bs(r.content, 'lxml')
soup

https://kind.krx.co.kr/corpgeneral/corpList.do
200


<html><body><section class="scrarea type-00">
<table class="list type-00 tmt30" summary="회사명, 업종, 주요제품, 상장일, 결산월, 대표자명, 홈페이지, 지역">
<caption>목록</caption>
<colgroup>
<col width="*"/>
<col width="15%"/>
<col width="20%"/>
<col width="11%"/>
<col width="8%"/>
<col width="10%"/>
<col width="8%"/>
<col width="12%"/>
</colgroup>
<thead>
<tr class="first" id="title-contents">
</tr>
</thead>
<tbody>
<tr class="first">
<td class="first" title="주식회사 씨엠티엑스"><img alt="코스닥" class="vmiddle legend" src="/images/common/icn_t_ko.gif"/> <a href="#companysum" id="companysum" onclick="companysummary_open('38821'); return false;" title="씨엠티엑스"> 씨엠티엑스</a> </td>
<td class="textOverflow" title="전자부품 제조업">전자부품 제조업</td>
<td class="textOverflow" title="반도체 장비용 소모성 부품">반도체 장비용 소모성 부품</td>
<td class="txc">2025-11-20</td>
<td class="txc">12월</td>
<td class="txc" title="박성훈, 박종화 (각자)">박성훈..</td>
<td class="txc">
<a class="btn ico homepage" href="http://cmtx.co.kr" target="_blank" title="http://cmtx.co.kr"><span>홈페이지 

In [4]:
int(soup.select_one(".info.type-00 > em").text.replace(",","")) // 100 +1

28

In [5]:
stock_list = soup.select("tbody > tr")
stock_list

[<tr class="first">
 <td class="first" title="주식회사 씨엠티엑스"><img alt="코스닥" class="vmiddle legend" src="/images/common/icn_t_ko.gif"/> <a href="#companysum" id="companysum" onclick="companysummary_open('38821'); return false;" title="씨엠티엑스"> 씨엠티엑스</a> </td>
 <td class="textOverflow" title="전자부품 제조업">전자부품 제조업</td>
 <td class="textOverflow" title="반도체 장비용 소모성 부품">반도체 장비용 소모성 부품</td>
 <td class="txc">2025-11-20</td>
 <td class="txc">12월</td>
 <td class="txc" title="박성훈, 박종화 (각자)">박성훈..</td>
 <td class="txc">
 <a class="btn ico homepage" href="http://cmtx.co.kr" target="_blank" title="http://cmtx.co.kr"><span>홈페이지 보기</span></a>
 </td>
 <td class="txc">경상북도</td>
 </tr>,
 <tr>
 <td class="first" title="더핑크퐁컴퍼니(주)"><img alt="코스닥" class="vmiddle legend" src="/images/common/icn_t_ko.gif"/> <a href="#companysum" id="companysum" onclick="companysummary_open('40385'); return false;" title="더핑크퐁컴퍼니"> 더핑크퐁컴퍼니</a> </td>
 <td class="textOverflow" title="영화, 비디오물, 방송프로그램 제작 및 배급업">영화, 비디오물, 방송프로그램 제작 및 배급

In [6]:
# 증권종류
stock_name = soup.select_one(".vmiddle.legend")['alt']

In [7]:
# 회사명
company_name = soup.select_one("#companysum").text

In [8]:
# 종목코드
code = soup.select_one("#companysum")['onclick'].split("'")[1]
code = f"{code:0<6}"
code

'388210'

In [9]:
# 업종
industry = soup.select_one("td:nth-child(2)").text
industry

'전자부품 제조업'

In [10]:
# 주요제품
main_products = soup.select_one("td:nth-child(3)").text
main_products

'반도체 장비용 소모성 부품'

In [11]:
# 상장일
listing_date = soup.select_one("td:nth-child(4)").text
listing_date

'2025-11-20'

In [12]:
# 결산월
fiscal_month = soup.select_one("td:nth-child(5)").text

In [13]:
# 대표자명
ceo_name = soup.select_one("td:nth-child(6)").text

In [14]:
# 홈페이지
website = soup.select_one(".btn.ico.homepage")['href']

In [15]:
# 지역
region = soup.select_one("td:nth-child(8)").text

In [16]:
result = {}
keys = ['주식종목', '회사명', '종목코드', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역']

for idx, stock in enumerate(soup):
    stock_name = stock.select_one(".vmiddle.legend")['alt']
    company_name = stock.select_one("#companysum").text
    code = stock.select_one("#companysum")['onclick'].split("'")[1]
    code = f"{code:0<6}"
    industry = stock.select_one("tbody > tr.first > td:nth-child(2)").text
    main_products = stock.select_one("tbody > tr.first > td:nth-child(3)").text
    listing_date = stock.select_one("tbody > tr.first > td:nth-child(4)").text
    fiscal_month = stock.select_one("tbody > tr.first > td:nth-child(5)").text
    ceo_name = stock.select_one("tbody > tr.first > td:nth-child(6)").text
    website = stock.select_one(".btn.ico.homepage")['href'] if stock.select_one(".btn.ico.homepage")['href'] != None else "없음"
    region = stock.select_one("tbody > tr.first > td:nth-child(8)").text
    
    values = (stock_name, company_name, code, industry, main_products, listing_date, fiscal_month, ceo_name, website, region)
    
    for key, value in zip(keys, values):
        result.setdefault(key, []).append(value)
        
result
df = pd.DataFrame(result)
df

,주식종목,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,코스닥,씨엠티엑스,388210,전자부품 제조업,반도체 장비용 소모성 부품,2025-11-20,12월,박성훈..,http://cmtx.co.kr,경상북도


# 전체 페이지

In [17]:
result = {}
for page in range(1,29):
    
    print(f"{page}/28", end="\r")
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList",pageIndex=page,currentPageSize=100,orderMode=3,orderStat="D",searchType=13, fiscalYearEnd="all", location="all")
    r = requests.post(url, data = payload)
    soup = bs(r.content, 'lxml')
    
    stock_list = soup.select("table.list.type-00.tmt30 > tbody > tr")

    keys = ['주식종목', '회사명', '종목코드', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역']

    for idx, stock in enumerate(stock_list):
        stock_name = stock.select_one(".vmiddle.legend")['alt']
        company_name = stock.select_one("#companysum").text
        code = stock.select_one("#companysum")['onclick'].split("'")[1]
        code = f"{code:0<6}"
        industry = stock.select_one("td:nth-child(2)").text 
        main_products = stock.select_one("td:nth-child(3)").text
        listing_date = stock.select_one("td:nth-child(4)").text
        fiscal_month = stock.select_one("td:nth-child(5)").text
        ceo_name = stock.select_one("td:nth-child(6)").text
        website = stock.select_one(".btn.ico.homepage")['href'] if stock.select_one(".btn.ico.homepage") != None else "없음"
        region = stock.select_one("td:nth-child(8)").text

        values = (stock_name, company_name, code, industry, main_products, listing_date, fiscal_month, ceo_name, website, region)

        for key, value in zip(keys, values):
            result.setdefault(key, []).append(value)
        
result
df = pd.DataFrame(result)
df

,주식종목,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,코스닥,씨엠티엑스,388210,전자부품 제조업,반도체 장비용 소모성 부품,2025-11-20,12월,박성훈..,http://cmtx.co.kr,경상북도
1,코스닥,더핑크퐁컴퍼니,403850,"영화, 비디오물, 방송프로그램 제작 및 배급업","콘텐츠, 라이선스, MD, 기타",2025-11-18,12월,김민석,없음,서울특별시
2,코넥스,아이엘커누스,403810,전자부품 제조업,IoT 스마트제어솔루션,2025-11-18,12월,지정석,http://www.conus.kr/,제주특별자치도
3,코스닥,그린광학,0015G0,사진장비 및 광학기기 제조업,초정밀 광학 시스템 및 광학 소재,2025-11-17,12월,조현일,http://www.greenoptics.com/,충청북도
4,코스닥,알트,459550,통신 및 방송 장비 제조업,"시니어폰 및 키즈폰, 셋탑박스",2025-11-17,12월,이상수,http://www.altech.kr/,경기도
...,...,...,...,...,...,...,...,...,...,...
2763,유가증권,유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
2764,유가증권,CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2765,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
2766,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


In [18]:
result = {}    
cols = ['주식종목', '회사명', '종목코드', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역']

page = 1
total_page = 1

while page <= total_page:
    print(f"{page}/{total_page}", end="\r")
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList",pageIndex=page,currentPageSize=100,orderMode=3,orderStat="D",searchType=13, fiscalYearEnd="all", location="all")
    r = requests.post(url, data = payload)
    # print(r.status_code)
    soup = bs(r.content, 'lxml')
    if total_page == 1:
        total_page = int(soup.select_one(".info.type-00 > em").text.replace(",","")) // 100 +1

    page_result = info_extraction(soup.select("tbody > tr"))
    for items in page_result:
        for key, value in zip(cols, items):
            result.setdefault(key, []).append(value)
            
    page += 1
    time.sleep(1)
        
df = pd.DataFrame(result)
df

,주식종목,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,코스닥,씨엠티엑스,388210,전자부품 제조업,반도체 장비용 소모성 부품,2025-11-20,12월,박성훈..,http://cmtx.co.kr,경상북도
1,코스닥,더핑크퐁컴퍼니,403850,"영화, 비디오물, 방송프로그램 제작 및 배급업","콘텐츠, 라이선스, MD, 기타",2025-11-18,12월,김민석,없음,서울특별시
2,코넥스,아이엘커누스,403810,전자부품 제조업,IoT 스마트제어솔루션,2025-11-18,12월,지정석,http://www.conus.kr/,제주특별자치도
3,코스닥,그린광학,0015G0,사진장비 및 광학기기 제조업,초정밀 광학 시스템 및 광학 소재,2025-11-17,12월,조현일,http://www.greenoptics.com/,충청북도
4,코스닥,알트,459550,통신 및 방송 장비 제조업,"시니어폰 및 키즈폰, 셋탑박스",2025-11-17,12월,이상수,http://www.altech.kr/,경기도
...,...,...,...,...,...,...,...,...,...,...
2763,유가증권,유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
2764,유가증권,CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2765,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
2766,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


In [19]:
df.to_csv("./data/stock_list.csv", index=False, encoding="utf-8-sig")

In [20]:
from io import StringIO

In [21]:
df_list = []
for page in range(1,28):
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList",pageIndex=page,currentPageSize=100,orderMode=3,orderStat="D",searchType=13, fiscalYearEnd="all", location="all")
    r = requests.post(url, data = payload)
    soup = bs(r.content, 'lxml')
    temp = pd.read_html(StringIO(r.text))
    df_list.append(temp[0])

df2=pd.concat(df_list)


In [22]:
df2.columns = ['회사명', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역']
df2

,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,씨엠티엑스,전자부품 제조업,반도체 장비용 소모성 부품,2025-11-20,12월,박성훈..,홈페이지 보기,경상북도
1,더핑크퐁컴퍼니,"영화, 비디오물, 방송프로그램 제작 및 배급업","콘텐츠, 라이선스, MD, 기타",2025-11-18,12월,김민석,NaN,서울특별시
2,아이엘커누스,전자부품 제조업,IoT 스마트제어솔루션,2025-11-18,12월,지정석,홈페이지 보기,제주특별자치도
3,그린광학,사진장비 및 광학기기 제조업,초정밀 광학 시스템 및 광학 소재,2025-11-17,12월,조현일,홈페이지 보기,충청북도
4,알트,통신 및 방송 장비 제조업,"시니어폰 및 키즈폰, 셋탑박스",2025-11-17,12월,이상수,홈페이지 보기,경기도
...,...,...,...,...,...,...,...,...
95,삼성제약,의약품 제조업,"의약품(까스명수,쓸기담,에프킬라에어졸,액체우황청심원) 제조,도매",1975-07-04,12월,정성택..,홈페이지 보기,경기도
96,삼성화재해상보험,보험업,"손해보험의 원수,재보험,운용자산의 투자활동",1975-06-30,12월,이문화,홈페이지 보기,서울특별시
97,삼양식품,기타 식품 제조업,"라면,스낵,유제품,장유,식용유,사료,향신료,건강음료(루이보스 티) 제조,도매",1975-06-30,12월,김정수..,홈페이지 보기,서울특별시
98,한화손해보험,보험업,손해보험,1975-06-30,12월,나채범,홈페이지 보기,서울특별시
